# Setup of SageMaker Endpoint for DescriptiveWorld computer vision model

Authored by Blair Jones, 2021.11.01

Model built using yolov5

Code inspired by AWS-provided example at: https://aws.amazon.com/blogs/machine-learning/speed-up-yolov4-inference-to-twice-as-fast-on-amazon-sagemaker/



### Instructions

Before running the notebook, install yolov5 in the Sagemaker root folder using the yolov5 instructions at:  https://github.com/ultralytics/yolov5

and run "aws configure" at command line to setup aws CLI.

### Setup

In [138]:
import numpy as np
import time
import json
import requests
import boto3
import os
import sagemaker
from tqdm.notebook import tqdm
from PIL import Image
import io

from sagemaker.pytorch.model import PyTorchModel
from sagemaker.predictor import Predictor

In [139]:
from sagemaker import get_execution_role
from sagemaker.session import Session

role = get_execution_role()
sess = Session()
region = sess.boto_region_name
bucket = 's3://descriptiveworld-models/CV_Models/'

In [140]:
print(region)
print(bucket)

us-west-2
s3://descriptiveworld-models/CV_Models/


In [141]:
# ensure that environment chosen for pytorch version >= 1.7 (as of 2021.11.01)

In [142]:
import torch
print(torch.__version__)

1.7.1


In [143]:
import sys
print(sys.version)

3.6.13 | packaged by conda-forge | (default, Feb 19 2021, 05:36:01) 
[GCC 9.3.0]


### Prepare model and code files for packaging

In [144]:
!aws s3 cp s3://descriptiveworld-models/CV_Models/df2_11_large_20211021/weights/best.pt ../../yolov5/

download: s3://descriptiveworld-models/CV_Models/df2_11_large_20211021/weights/best.pt to ../../yolov5/best.pt


In [145]:
%cd ~/SageMaker/yolov5/

/home/ec2-user/SageMaker/yolov5


In [146]:
ls

bcj_export.py        current.torchscript.pt  model.pt           setup.cfg
bcj_sm_ep_detect.py  data/                   models/            train.py
best.pt              detect.py               model.tar.gz       tutorial.ipynb
blank.py             Dockerfile              README.md          utils/
code/                export.py               requirements.txt*  val.py
CONTRIBUTING.md      hubconf.py              runs/              yolov5/
currentmodel.tar.gz  LICENSE                 s3:/


In [147]:
# it is not necessary to convert to torchscript : this was done initially based on vague webpage reference
#!python ./bcj_export.py --weights ./best.pt

In [148]:
!cp best.pt model.pt

In [149]:
!mkdir ./code

mkdir: cannot create directory ‘./code’: File exists


## Run everything below this line every time the yolov5 source code changes
<div style='color:red'> Run everything below this line every time the yolov5 source code changes </div>

In [150]:
!cp ./utils/ -r ./code/utils/
!cp ./models/ -r ./code/models/
!cp ./data/ -r ./code/data/
#!cp ./requirements.txt ./code/ # this file was customized to include special libraries needed by DW

## Run everything below this line every time the Endpoint source code changes
<div style='color:red'> 
    <p>Run everything below this line every time the Endpoint source code changes </p>
    <p>Before redeploying the endpoint, use the Sagemaker console to delete:</p>
        
        - Endpoint
        
        - Endpoint Configuration
        
        - Model
        


In [157]:
# the primary version of the code is in the github repo location
# ensure that the latest copy of that file has been copied to the main folder under yolov5
!cp ../descriptiveworld/CV/bcj_sm_ep_detect.py  .
!cp ./bcj_sm_ep_detect.py  ./code/

In [158]:
# saves model archive to S3
!tar -czvf ./model.tar.gz ./model.pt
!aws s3 cp ./model.tar.gz s3://descriptiveworld-models/CV_Models/

./model.pt
upload: ./model.tar.gz to s3://descriptiveworld-models/CV_Models/model.tar.gz


### Create the model and endpoint

In [159]:
model_archive = './model.tar.gz'
prefix = 's3://descriptiveworld-models/CV_Models'
model_path = sess.upload_data(path=model_archive, key_prefix=prefix)
model_path

's3://sagemaker-us-west-2-769212126689/s3://descriptiveworld-models/CV_Models/model.tar.gz'

### Create Model on SageMaker Model with DescriptiveWorld artifacts

In [160]:
%%time
# reference:  https://stackoverflow.com/questions/68150444/aws-sagemaker-fails-loading-pytorch-pth-weights
# reference:  https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/pytorch/model.py
# reference:  https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/model.py

framework_version = '1.7.1'
py_version = 'py3'

sm_model = PyTorchModel(model_data=model_path,
                       framework_version=framework_version,
                       role=role,
                       sagemaker_session=sess,
                       entry_point='bcj_sm_ep_detect.py',
                       source_dir='code',
                       py_version=py_version,
                       env={"COMPILEDMODEL": 'False', 'MMS_MAX_RESPONSE_SIZE': '100000000', 'MMS_DEFAULT_RESPONSE_TIMEOUT': '500'})

CPU times: user 62 µs, sys: 8 µs, total: 70 µs
Wall time: 74.1 µs


### Deploy / Update Endpoint (will create a SageMaker model at same time)

As of 17 Nov 2021, the update method does not overwrite the existing endpoint, so recommended usage is to delete the existing endpoint and deploy a new one

There are three things that need to be updated/deleted:

* SageMaker Inference Model (if the endpoint code, model or code dependencies have changed)
* Endpoint Configuration (otherwise the old version will continue to reference old code/model artifacts)
* Endpoint (which points to the other 2 items)

In [161]:
%%time
instance_type = 'ml.t2.medium'
endpoint_name = 'descriptiveworld-SageMaker-EndPoint'

# initial deployment of endpoint
dw_predictor = sm_model.deploy(initial_instance_count=1, 
                               instance_type=instance_type,
                               endpoint_name=endpoint_name
                                )

# update an existing endpoint
#sagemaker_session = empty_sagemaker_session()
# create predictor object
#dw_predictor = Predictor(endpoint_name=endpoint_name, sagemaker_session=sagemaker_session)

# update endpoint
#predictor.update_endpoint(
#    model_name=sm_model.name,
#    instance_type=instance_type, 
#    initial_instance_count=1
#)


print()
print(sm_model.name)
print(dw_predictor.endpoint_name)

-------------!
pytorch-inference-2021-11-23-18-01-20-230
descriptiveworld-SageMaker-EndPoint
CPU times: user 7.2 s, sys: 742 ms, total: 7.94 s
Wall time: 6min 40s


### Test Endpoint

In [162]:
%%time
client = boto3.client('sagemaker-runtime', region_name=region)
#sample_img_url = "https://d2ph5fj80uercy.cloudfront.net/04/cat1600.jpg"
#sample_img_url = "https://media.gq.com/photos/60f9c697101cc04fad71e5cf/master/pass/BEST-BASICS-1.jpg" #TOO LARGE
sample_img_url =  "https://c.pxhere.com/photos/9e/86/blue_vest_fashion_male_shirt-1059471.jpg!s"
content_type='JPEG'
image_data = requests.get(sample_img_url).content

# resize image to max 640x640 px - otherwise image size will cause fatal error for JSON output size
im_resize = Image.open(io.BytesIO(image_data))
im_resize.thumbnail((640,640))
buf = io.BytesIO()
im_resize.save(buf, format='JPEG')
byte_im = buf.getvalue()

response = client.invoke_endpoint(EndpointName=dw_predictor.endpoint_name, Body=byte_im, ContentType=content_type)
pred_out = response['Body'].read().decode()

CPU times: user 38.8 ms, sys: 0 ns, total: 38.8 ms
Wall time: 7.41 s


In [163]:
print('length =', len(pred_out), '\n')
print(pred_out)

length = 532 

{"source-ref": "TBD", "num-detected-objects": 1, "bounding-box-attribute-name": {"image_size": [{"width": 255, "height": 340, "depth": 3}], "annotations": [{"class_id": 2, "left": 18, "top": 14, "width": 210, "height": 294}]}, "bounding-box-attribute-name-metadata": {"objects": [{"confidence": 0.45}], "class-map": {"2": "long sleeve top"}, "type": "descriptiveworld/object-detection", "human-annotated": "no", "creation-date": "2021-11-23 18:08:23.859244", "job-name": "descriptive_world_identify_garments"}, "original-image": ""}


### Run Inference

In [ ]:
%%time
iters = 1000
warmup = 100
client = boto3.client('sagemaker-runtime', region_name=region)

content_type = 'application/x-image'

sample_img_url = "https://github.com/ultralytics/yolov5/raw/master/data/images/zidane.jpg"
body = requests.get(sample_img_url).content

dw_perf = []
  
for i in tqdm(range(iters)):
    t0 = time.time()
    response = client.invoke_endpoint(EndpointName=dw_predictor.endpoint_name, Body=body, ContentType=content_type)
    t1 = time.time()
    #convert to millis
    dw_elapsed = (t1-t0)*1000
    
    if warmup == 0:
        dw_perf.append(uncompiled_elapsed)
    else:
        print(f'warmup ({i}, {iters}) : dw - {dw_elapsed} ms')
        warmup = warmup - 1